# Coexistence base library

    Ben Freasier
    September 2020
    
The purpose of this script is to develop fitting functions to the various versions of the van der Waals models promulgated in the [Generalized van der Waals Theory of Moleclar Fluids in Bulk at at Surfaces](elsevier.com/books/generalized-van-der-waals-theory-of-molecular-fluids-in-bulk-and-at-surfaces/nordholm/978-0-12-811136-9) (GVDWB). 

For the classical van der Waals and the simple generalized van der Waals (GVDW(S)) equations of state we will calculate the coexistence curves using [Parametric solution of the van der Waals liquid-vapor coexistence curve](https://www.wgtn.ac.nz/scps/about/staff/pdf/Van-der-Waals-liquid-vapor-coexistence-curve.pdf) by John Lekner. I will refer to this reference as JL.

The other more complicated generalized van der Waals models in GVDWB may not be so susceptiple to Lekner's parametric analysis. In that case we will calculate the coexistence curve using the a more brute force numerical integration for the Maxwell construction.

Once the coexistence curves have been calculated, the curves will be fitted with spline functions in order to optimize calculation time in future applications.

Priority List (from Sture) 

- vdW   the old workhorse (DONE) and 
- GvdW(S) improving the dense fluid behaviour; (DONE)
- GvdW(HS-B2) as the simplest attempt to be OK at high and low density:
- GvdW(I) and 
- GvdW(CS) which do a much better job of representing density dependent excluded volume; (DONE)
- GvdW(VHS-I) which shows how hole creation is coupled to both entropy and binding energy;

All the others are of more exploratory interest.

Here is the code:




<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#classic_vdw()" data-toc-modified-id="classic_vdw()-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>classic_vdw()</a></span></li><li><span><a href="#cubic(s)" data-toc-modified-id="cubic(s)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>cubic(s)</a></span></li><li><span><a href="#pmu_maxwell()" data-toc-modified-id="pmu_maxwell()-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>pmu_maxwell()</a></span></li><li><span><a href="#GVDW(CS)" data-toc-modified-id="GVDW(CS)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>GVDW(CS)</a></span><ul class="toc-item"><li><span><a href="#cs_vdw()" data-toc-modified-id="cs_vdw()-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>cs_vdw()</a></span></li></ul></li><li><span><a href="#GvdW(I)" data-toc-modified-id="GvdW(I)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>GvdW(I)</a></span><ul class="toc-item"><li><span><a href="#gvdwi()" data-toc-modified-id="gvdwi()-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>gvdwi()</a></span></li></ul></li><li><span><a href="#GvdW(HS-B2)" data-toc-modified-id="GvdW(HS-B2)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>GvdW(HS-B2)</a></span><ul class="toc-item"><li><span><a href="#gvdwhsb2()" data-toc-modified-id="gvdwhsb2()-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>gvdwhsb2()</a></span></li></ul></li><li><span><a href="#GvdW(VHS-I)" data-toc-modified-id="GvdW(VHS-I)-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>GvdW(VHS-I)</a></span><ul class="toc-item"><li><span><a href="#oev()" data-toc-modified-id="oev()-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>oev()</a></span></li><li><span><a href="#gvdw_vhsi()" data-toc-modified-id="gvdw_vhsi()-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>gvdw_vhsi()</a></span></li></ul></li><li><span><a href="#GvdW(S)-corr" data-toc-modified-id="GvdW(S)-corr-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>GvdW(S)-corr</a></span><ul class="toc-item"><li><span><a href="#gvdws_corr()" data-toc-modified-id="gvdws_corr()-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>gvdws_corr()</a></span></li></ul></li></ul></div>

## Imports
We will import cosh, sinh, and arange from [numpy](numpy.org) so we can use array operations. 

We will import plt from [matplotlib](matplotlib.org) so we can plot  our results.

We will use [namedtuples](https://docs.python.org/3/library/collections.html) as a simplified class for holding coexistence data.

We will use [scipy.interpolate.interp1d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) to define our fit as a spline fit.

In [1]:
from numpy import cosh, sinh, arange, pi, cumsum, dot, log, shape, multiply, array, sqrt
import matplotlib.pyplot as plt
from collections import namedtuple
from scipy.interpolate import interp1d
from scipy.optimize import fsolve 
import scipy.integrate as si
from scipy.optimize import curve_fit
import warnings

## classic_vdw()

This function defines the classic van der Waals equation of state with molecular exclusion volume $\sigma$:

$$
P = {Nk_BT\over V - {2\pi\over3}\sigma^3} - {a\over V^2}
$$

Inputs:

- rho = density = `N/V` reduced by $\sigma^3$. It can be an array.
- t = reduced by the van der Waals attraction, $a$. Since we are working on an isotherm in this study, it will usually be parameterized with in a higher level function that will appear to be only a function of the density.

Output

- pressure in units of $\sigma^3/a$



In [2]:
def classic_vdw(rho,t=1.):
  v = 1./rho
  p = t/(v-2.*pi/3)-1.*rho*rho
  return p

## cubic(s)

This function takes the classic cubic vdw equations and fits the coexistence curve to linear spline functions. The motivation for this is to speed up the execution of the coexistence curve calculation when it is being used in a dynamic situation (eg, a student tutorial app).

The logic of the program is

- calculate the coexistent gas and liquid densities (in units scaled to the critical point) for each value in the passed `s` array.

- plot those values. I tried to use the minimum number of `s` points to capture the shape of the curve. I was happy with 9 points.

- I then did a spline fit for both the gas and liquid branches.

- I then defined a range of densities over which to interpolate the gas and liquid coexistent temperatures.

- I then defined a named tuple containing the spline functions for the gas and liquid branch for use in other applications.

INPUT: s, array of JLs `s/2` values

OUTPUT: coex, named tuple of type Coex containing spline fit functions

In [3]:
def cubic(s,doplot=False):
  f=(s*cosh(s)-sinh(s))/(sinh(s)*cosh(s)-s)
  c = cosh(s)
  g = 1 + 2*c*f + f*f
  t = (27./4.)*f*(c+f)/(g*g)
  p = 27*f*f*(1-f*f)/(g*g)
  ns = 3*f*(c+f)/g
  nd = (6*f/g)*sinh(s)
  nl = (2*ns + nd)/2
  ng = nl-nd
  if doplot:
    plt.plot(ng,t,"o",c="k",label="Exact values")
    plt.plot(nl,t,"o",c="k")

  k = "slinear"
  fg = interp1d(ng,t,kind=k,fill_value="extrapolate")
  x = arange(ng[-1],ng[0],.001)
  y = fg(x)
  if doplot:
    plt.plot(x,y,label="gas branch fit")

  fl = interp1d(nl,t,kind=k,fill_value="extrapolate")
  x = arange(nl[0],nl[-1],.01)
  y = fl(x)
  if doplot:
    plt.plot(x,y,label="liquid branch fit")

    plt.legend()
    plt.xlabel("n*")
    plt.ylabel("T*")
    plt.show()
  
  Coex = namedtuple('Coex', "fg fl".split())
  coex = Coex(fg,fl)
  return coex

# s = [.001,.25,.5,1.,1.5,2.,3.,5.,10.,200.]
# cubres = cubic_vdw=cubic(s,doplot=True)

## pmu_maxwell()

One way of doing the Maxwell construction is to to calculate isotherms of the pressure $P$ and the chemical potential, $\mu$, and note that below the critical temperature that when $\mu$ is expressed as a function of $P$ that the liquid branch of $\mu$ intersects with the gas branch of $\mu$ that marks the phase coexistence.

Firstly, we need to derive the chemical potential from the equation of state. The fundamental equation is

$$ P = -\left(\
\partial A\over \partial V \right)_{N,T} \tag{1}.$$

It is more convenient to talk about excess quantitities over the ideal gas so that

$$ P - P_{id} = -\left(\
\partial (A-A_{id})\over \partial V \right)_{N,T} \tag{2}.$$

We know that the free energy of an ideal gas , $A_{id}$, is given by

$$
A_{id}(N,V,T) = -Nk_BT\left[\log\left({V\over N\lambda^3}\right)+1 \right]\tag{3},
$$

where $\lambda$ is the thermal wavelength and doesn't depend on the density. Since we are going to working on as isotherm when finding coexistent chemical potentials, we can ignore constant additive factors to the free energy. So, we can write our reference density dependent ideal free energy as  

$$
A_{ref}(N,V,T) = -Nk_BT\left[\log\left({V\over N}\right)\right]\tag{4},
$$

So, we do the integration


$$ 
A(N,T,V)\tag{5} =
-\int_\infty^V\,dV^\prime \left(P-{{Nk_BT}\over V^\prime}\right) + A_{ref}(N,V,T) 
$$

We now make a change of variables of the density, $\rho^\prime = N/V^\prime$, so that

$$ dV^\prime = -{N\over {\rho^\prime}^2}d\rho^\prime
\tag{6}.$$

Combining (5) and (6) and reversing integral limits gives 

$$
{A(N,T,\rho)\over Nk_BT} = 
\int_0^\rho {1\over\rho^\prime} \left({{P\over \rho^\prime k_B T}-1}\right) 
d\rho^\prime + \log(\rho)  \tag{7}.
$$

Since $G = A + PV$, and $\mu = G/N$, we can express the chemical potential over ideal on an isotherm as

$$
{\mu^{ex}\over k_BT} = 
\int_0^\rho {1\over\rho^\prime}\left({{P\over \rho^\prime k_B T}-1}\right)
d\rho^\prime + \log(\rho) + {P\over k_BT\rho}. \tag{8}
$$





We will use Equation 8 is what we will use for doing the Maxwell construction for a an equation of state on an isotherm. The calling sequence is

    pmu_maxwell(pfunc,t,rng=[.0001,.3,.001],plot=[])

where

`pfunc`: equation of state

`t`: temperature

`rng`: [rhomin,rhodmax,delta_rho]

`plot`: list of plot indices for which a plot will be generated for each index in the list. These indices are:

  - 0 - equation of state
  - 1 - compressibility factor
  - 2 - integrand in Equation 8
  - 3 - integral in Equation 8
  - 4 - P-$\mu$ 

It returns a tuple of (temperature, gas density, liquid density)

What the function does:

  - calculates eos over specified density range on isotherm
  - calculates compressibility factor
  - calculates integrand in Eqn 8
  - calculates integral in Eqn 8
  - calculates $\mu$
  - finds density ranges of liquid and gas branch
  - fits each branch to a linear splne function set
  - finds the interesection of the two branches to determine the coexistent pressure
  - determines the coexistent densities from the coexistent pressure and equation of state
  - returns the temperature and coexistent densities

In [4]:
def pmu_maxwell(pfunc,t,rng=[.0001,.3,.001],plot=[]):

  # calculate eos over specified density range on isotherm
  r = arange(rng[0],rng[1],rng[2])
  eos = lambda x: pfunc(x,t=t)
  p=eos(r)
  if 0 in plot:
    plt.plot(r,p)
    plt.xlabel("$\\rho\\sigma^3$",size=14)
    plt.ylabel("$P\\sigma^3/a$",size=14)
    plt.show()
    plt.clf()

  # calculate compressibility factor
  cf = lambda x: (eos(x)/(x*t))
  if 1 in plot:
    y = cf(r)
    plt.plot(r,y)
    plt.xlabel("$\\rho\sigma^3$",size=14)
    plt.ylabel("$Z$",size=14)
    plt.show()
    plt.clf()

  # calculate integrand in Eqn 8
  intg = lambda x: (cf(x)-1)/x
  if 2 in plot:
    y = intg(r)
    plt.plot(r,y)
    plt.xlabel("$\\rho\sigma^3$",size=14)
    plt.ylabel("A integrand",size=14)
    plt.show()
    plt.clf()

  # calculate integral in Eqn 8
  pieces = [si.quad(intg, r[i], r[i+1])[0] for i in range(len(r)-1)]
  yy = cumsum([0] + pieces)
  if 3 in plot:
    # plt.plot(r,y)
    # plt.plot(r,yy)
    plt.plot(r,y)
    plt.xlabel("$\\rho\sigma^3$",size=14)
    plt.ylabel("Integral",size=14)
    plt.show()
    plt.clf()

  # calculate mu
  v = 1./r
  mu = multiply(p,v)/t + yy + log(r)

  # find density ranges of liquid and gas branch
  for i in range(len(r)-1): 
    if mu[i] > mu[i+1]:
      break
  ng = i
  for i in range(len(r)-1,1,-1): 
    if mu[i] < mu[i-1]:
      break
  nl = i

  # fit each branch to a linear splne function set
  k = "slinear"
  fg = interp1d(p[:ng],mu[:ng],kind=k,fill_value="extrapolate")
  fl = interp1d(p[nl:],mu[nl:],kind=k,fill_value="extrapolate")

  # find the interesection of the two branches to determine the coexistent pressure
  f = lambda x: fg(x)-fl(x)
  ptrial = (p[ng]+p[nl])/2.
  z = fsolve(f,ptrial)
  pcoex = z[0]
  mucoex = fg(pcoex)

  # determine the coexistent densities from the coexistent pressure and equation of state
  f = lambda x: eos(x)-pcoex
  r1 = fsolve(f,r[0])[0]
  f = lambda x: (eos(x)-pcoex)/(x-r1)
  r2 = fsolve(f,r[0])[0]
  f = lambda x: (eos(x)-pcoex)/(x-r1)/(x-r2)
  r3 = fsolve(f,r[0])[0]
  cd = [r1,r2,r3]
  cd.sort()

  # plot coexistence curve
  if 4 in plot:
    plt.plot(p[:ng],mu[:ng],"b",label="gas branch")
    plt.plot(p[nl:],mu[nl:],"g",label="liquid branch")
    plt.plot(p[ng:nl],mu[ng:nl],"r",ls = "--",label="unstable")
    plt.plot([pcoex],[mucoex],"o",label="coexistance")
    plt.xlabel("$P$",size=14)
    plt.ylabel("$\mu/(k_BT)$",size=14)
    # plt.xlim(.006,.008)
    # plt.ylim(-2.35,-2.24)
    plt.legend()
    plt.show()
    plt.clf

  # returns the temperature and coexistent densities
  return t,cd[0],cd[2]

# z=pmu_maxwell(classic_vdw,.06,rng=[.001,.45,.001],plot=[5])
# print(z)

## GVDW(CS)

The Carnahan-Starling van der Waals equation of state is given by 

$$
P = Nk_BT {(1 + \eta + \eta^2-\eta^3)\over(1-\eta)^3}
 - {a\over V^2}, \tag{9}
$$

where $\eta=\pi\sigma^3/6$. This is implemented in code in the next section.

### cs_vdw()

This has been optimized for array operations. The temperature is scaled to $a$ and the length scale is $\sigma$.

In [5]:
def cs_vdw(rho,t=.1):
  e = pi*rho/6.
  e2 = multiply(e,e)
  e3 = multiply(e2,e)
  x = 1.-e
  x2 = multiply(x,x)
  x3 = multiply(x2,x)
  p = t*(1+e+e2-e3)/x3-multiply(rho,rho)
  return p

## GvdW(I)

The GvdW(I) EOS is given by 

$$
P = {k_BT\over v}\left[1+{2\pi\over3}{\sigma^3\over v-\sigma^3}\right]-{a\over v^2}\tag{10}
$$


### gvdwi()

The units ae scaled with $\sigma$ and $a$.

In [6]:
def gvdwi(rho,t=1.):
  v = 1./rho
  p = (t/v)*(1+(2./3.)*pi/(v-1))-1.*rho*rho
  return p

## GvdW(HS-B2)

The GvdW(HS-B2) EOS is given by 

$$
P = {k_BT\over v-\sigma}+{k_BT((2\pi/3)-1)\sigma^3-a\over v^2}\tag{11}
$$

### gvdwhsb2()

So, in units of $T/(ak_B)$ and $\rho\sigma^3$ and $P\sigma^3/a$

In [7]:
def gvdwhsb2(rho,t=1.):
  v = 1./rho
  c = 2.*pi/3.-1.
  p = t/(v-1) + (t*c-1)*rho*rho
  return p

## GvdW(VHS-I)

The EOS is given  by

$$
P = {k_BT\over v} + {2\pi\over 3}{k_BT\over v}{d^3\over v-d^3}
    -{8\pi\epsilon\sigma^3\over v^2}
    \left({\sigma^3\over 3 d^3}-{\sigma^9\over 9d^9}\right)\tag{12}
$$

**Note that relation in the book (Eqn 6.43) is missing an extra factor of $v$ in the denominator of
the last term.**

so our function is scaled by $\rho\sigma^3$, $k_BT/a$, and returns 
the pressure as $P\sigma^3/a$. d is the optimized excluded volume and is given by 

$$
b(x) = {2\over x}\left[\left(\sqrt{1+x}-1\right)\right]^{1/2}\tag{13}
$$

where $x= k_BT/\epsilon$ and returns the optimized excluded volume scaled to $\sigma^3$.

### oev()

In [8]:
def oev(x):
    z = sqrt(1+x)-1
    return sqrt((2/x)*z)

### gvdw_vhsi()

In [9]:
def gvdw_vhsi(rho,t=1.):
    v = 1./rho
    a = 16.*pi/9
    te = a*t
    d3 = oev(te) 
    p = (t/v)
    p += (2*pi/3.)*p*d3/(v-d3)
    p -= (8.*pi/(a*v*v))*(1./(3.*d3)-1./(9.*d3*d3*d3))
    return p

## GvdW(S)-corr

The GvdW(S)-corr EOS is given by 

$$
P = {k_BT\over v-\sigma^3}-{a+a_{corr}/T\over v^2},
$$

where 
$$ 
a= {16\pi\over9}\epsilon\sigma^3
$$

and

$$
a_{corr}/T = {128\pi\over315}{\epsilon\over k_BT} \epsilon\sigma^3 = {8\over35}{a\over T}.
$$

### gvdws_corr()

In [10]:
a = 16*pi/9.
b = 128*pi/315.
acorr = b/a
# b/a,8./35.

def gvdws_corr(rho,t=1.):
  v = 1./rho
  p = t/(v-1.)-(1.+acorr/t)/(v*v)
  return p